In [11]:
import sys
import os

PACKAGE_PARENT = '..'
SCRIPT_DIR = os.path.abspath('')
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))

In [29]:
from bs4 import BeautifulSoup
import requests
import re
import numpy as np
import pandas as pd
import re
import pickle

import importlib
import utils
importlib.reload(utils)
from tqdm import tqdm

from utils import get_parsed, get_parsed_remove_comment, baseball_base_url

In [3]:
content = get_parsed_remove_comment(baseball_base_url + '/leagues/')

In [6]:
leagues = content.find(id='div_9783046937')

In [7]:
table = leagues.table

In [11]:
all_trs = table.find_all('tr')
column_names = [ele.text.strip() for ele in all_trs[0].find_all('td')]
column_names[0] = 'League Name'
column_names

['League Name', 'Wins', 'Batting', 'Pitching', 'Fielding', 'Misc']

In [60]:
data = []
for tr in all_trs[:8]:
    text = [ele.text.strip() for ele in tr.find_all('td')]
    temp = [ele.a['href'] for ele in tr.find_all('td')]
    data.append(text[:1] + temp[1:])
df = pd.DataFrame(data, columns=column_names)

In [61]:
df.head(2)

,League Name,Wins,Batting,Pitching,Fielding,Misc
0,National League,/leagues/NL/,/leagues/NL/bat.shtml,/leagues/NL/pitch.shtml,/leagues/NL/field.shtml,/leagues/NL/misc.shtml
1,American League,/leagues/AL/,/leagues/AL/bat.shtml,/leagues/AL/pitch.shtml,/leagues/AL/field.shtml,/leagues/AL/misc.shtml


In [62]:
df.to_csv(os.path.join('leagues_index', 'all_leagues.csv'))

## Scrape data now

In [8]:
df = pd.read_csv(os.path.join('leagues_index', 'all_leagues.csv'))

In [9]:
df

,Unnamed: 0,League Name,Wins,Batting,Pitching,Fielding,Misc
0,0,National League,/leagues/NL/,/leagues/NL/bat.shtml,/leagues/NL/pitch.shtml,/leagues/NL/field.shtml,/leagues/NL/misc.shtml
1,1,American League,/leagues/AL/,/leagues/AL/bat.shtml,/leagues/AL/pitch.shtml,/leagues/AL/field.shtml,/leagues/AL/misc.shtml
2,2,Federal League,/leagues/FL/,/leagues/FL/bat.shtml,/leagues/FL/pitch.shtml,/leagues/FL/field.shtml,/leagues/FL/misc.shtml
3,3,American Association,/leagues/AA/,/leagues/AA/bat.shtml,/leagues/AA/pitch.shtml,/leagues/AA/field.shtml,/leagues/AA/misc.shtml
4,4,Players League,/leagues/PL/,/leagues/PL/bat.shtml,/leagues/PL/pitch.shtml,/leagues/PL/field.shtml,/leagues/PL/misc.shtml
5,5,Union Association,/leagues/UA/,/leagues/UA/bat.shtml,/leagues/UA/pitch.shtml,/leagues/UA/field.shtml,/leagues/UA/misc.shtml
6,6,National Association,/leagues/NA/,/leagues/NA/bat.shtml,/leagues/NA/pitch.shtml,/leagues/NA/field.shtml,/leagues/NA/misc.shtml
7,7,All Major Leagues,/leagues/MLB/,/leagues/MLB/bat.shtml,/leagues/MLB/pitch.shtml,/leagues/MLB/field.shtml,/leagues/MLB/misc.shtml


In [18]:
def process(table):
    
    table_name = table.previous_sibling.previous_sibling.find('h2').text
    print(table_name)
    
    table_header = table.thead
    column_names = [th.text for th in table_header.find_all('th')]
    #print([th.attrs for th in table_header.find_all('th')])
    #print(column_names)
    columns_exp = [th.attrs['data-tip'] if 'data-tip' in th.attrs else '' for th in table_header.find_all('th')]
    columns_short_exp = [th.attrs['data-stat'] for th in table_header.find_all('th')]
    #print(columns_short_exp)
    result = []
    for tr in table.tbody.find_all('tr'):
        if 'class' not in tr.attrs:
            year = tr.find('th').text
            rest = [ele.text for ele in tr.find_all('td')]
            result.append([year] + rest)
    df_table = pd.DataFrame(result, columns=column_names)
    df_explain = pd.DataFrame([columns_short_exp, columns_exp], columns=column_names)
    return table_name, df_table, df_explain
    

In [23]:
folder = 'leagues_index'
for col_num, column_name in enumerate(['Wins', 'Batting', 'Pitching', 'Fielding', 'Misc']):
    for row_num in range(min(df.shape[0], 2)):
        content = get_parsed(baseball_base_url + df.loc[row_num, column_name])
        tables = content.find_all(attrs={'class':'table_outer_container'})
        for idd, table in enumerate(tables):
            table_name, df_table, df_explain = process(table)
            file_name = (f'{row_num:02}_' + df.loc[row_num, 'League Name'] + '_' + column_name 
                         + '_' + table_name + '_' + str(idd) + '.csv')
            file_name = re.sub(' ', '_', file_name)
            #print(file_name)
            df_table.to_csv(os.path.join(folder, file_name))
            file_name_exp = (f'{row_num:02}_' + df.loc[row_num, 'League Name'] + '_' + column_name 
                         + '_' + table_name + '_' + str(idd) + '_' + 'explanation' + '.csv')
            file_name_exp = re.sub(' ', '_', file_name_exp)
            #print(file_name_exp)
            df_explain.to_csv(os.path.join(folder, file_name_exp))

In [27]:
folder = 'temp'
all_dfs = []
for col_num, column_name in enumerate(['Wins', 'Batting', 'Pitching', 'Fielding', 'Misc']):
    for row_num in range(min(df.shape[0], 2)):
        content = get_parsed(baseball_base_url + df.loc[row_num, column_name])
        tables = content.find_all(attrs={'class':'table_container'})
        print(len(tables))
        for idd, table in enumerate(tables):
            table_name, df_table, df_explain = process(table)
            file_name = (f'{row_num:02}_' + df.loc[row_num, 'League Name'] + '_' + column_name 
                         + '_' + table_name + '_' + str(idd) + '.csv')
            file_name = re.sub(' ', '_', file_name)
            #print(file_name)
            df_table.to_csv(os.path.join(folder, file_name))
            file_name_exp = (f'{row_num:02}_' + df.loc[row_num, 'League Name'] + '_' + column_name 
                         + '_' + table_name + '_' + str(idd) + '_' + 'explanation' + '.csv')
            file_name_exp = re.sub(' ', '_', file_name_exp)
            #print(file_name_exp)
            df_explain.to_csv(os.path.join(folder, file_name_exp))
            
            all_dfs.append([df.loc[row_num, 'League Name'], column_name, table_name, df_table, df_explain])

2
Team Wins
Team Wins
1
Team Wins
2
League Year-By-Year Batting--Averages
League Year-By-Year Batting--Totals
2
League Year-By-Year Batting--Averages
League Year-By-Year Batting--Totals
2
League Year-By-Year Pitching--Averages
League Year-By-Year Pitching--Totals
2
League Year-By-Year Pitching--Averages
League Year-By-Year Pitching--Totals
2
League Year-By-Year Fielding--Averages
League Year-By-Year Fielding--Totals
2
League Year-By-Year Fielding--Averages
League Year-By-Year Fielding--Totals
1
League Year-By-Year Miscellaneous
1
League Year-By-Year Miscellaneous


In [30]:
with open('pkl_all_league_index.pkl', 'wb') as f:
    pickle.dump(all_dfs, f)
with open('pkl_all_league_index.pkl', 'rb') as f:
    all_dfs = pickle.load(f)

In [59]:
wrong_count = 4
columns = ['Question', 'Correct'] + ['Wrong_' + str(i + 1) for i in range(wrong_count)] +  ['tags']
years_range = range(1980, 2021)
tags_ori = ['Totals']

question_list = []
import random
random.seed(1000)
for league, col, tab, df, df_exp in all_dfs:
    print(league + ' ' + col + ' ' + tab)
    if 'wins' in tab.lower():
        for i in range(df.shape[0]):
            year = int(df.loc[i, 'Year'])
            if year < 1980:
                break
            # each team wins that year
            for column in df.columns:
                if column != 'Year' and column != 'G':
                    question = f'How many games did {column} win in {year} in {league}?'
                    correct = int(df[df['Year'] == str(year)]['G'])
                    options = random.sample(list(range(int(correct * 0.6), int(correct * 1.4))), k = wrong_count + 1)
                    wrongs = [opt for opt in options if opt != correct][:wrong_count]
                    tags = tags_ori + []
                    question_list.append([question] + [correct] + wrongs + [tags])
    elif 'totals' in tab.lower() and 'batting' in tab.lower():
        for i in range(df.shape[0]):
            year = int(df.loc[i, 'Year'])
            if year < 1980:
                break
            # number of teams used that year
            question = f'How many teams participated in games in {year} in {league}?'
            correct = int(df[df['Year'] == str(year)]['Tms'])
            options = random.sample(list(range(int(correct * 0.6), int(correct * 1.4))), k = wrong_count + 1)
            wrongs = [opt for opt in options if opt != correct][:wrong_count]
            tags = tags_ori + []
            question_list.append([question] + [correct] + wrongs + [tags])
            
            # batters used that year
            question = f'How many batters participated in games in {year} in {league}?'
            correct = int(df[df['Year'] == str(year)]['#Bat'])
            options = random.sample(list(range(int(correct * 0.6), int(correct * 1.4))), k = wrong_count + 1)
            wrongs = [opt for opt in options if opt != correct][:wrong_count]
            tags = tags_ori + []
            question_list.append([question] + [correct] + wrongs + [tags])
            
            # batters used that year
            question = f'How many games are played/pitched in {year} in {league}?'
            correct = int(df[df['Year'] == str(year)]['G'])
            options = random.sample(list(range(int(correct * 0.6), int(correct * 1.4))), k = wrong_count + 1)
            wrongs = [opt for opt in options if opt != correct][:wrong_count]
            tags = tags_ori + []
            question_list.append([question] + [correct] + wrongs + [tags])
            
            # strikeouts used that year
            question = f'How many strikeouts () are in {year} in {league}?'
            correct = int(df[df['Year'] == str(year)]['SO'])
            options = random.sample(list(range(int(correct * 0.6), int(correct * 1.4))), k = wrong_count + 1)
            wrongs = [opt for opt in options if opt != correct][:wrong_count]
            tags = tags_ori + []
            question_list.append([question] + [correct] + wrongs + [tags])
            
            # stolen bases used that year
            question = f'How many stolen bases are in {year} in {league}?'
            correct = int(df[df['Year'] == str(year)]['SB'])
            options = random.sample(list(range(int(correct * 0.6), int(correct * 1.4))), k = wrong_count + 1)
            wrongs = [opt for opt in options if opt != correct][:wrong_count]
            tags = tags_ori + []
            question_list.append([question] + [correct] + wrongs + [tags])
    elif 'totals' in tab.lower() and 'fielding' in tab.lower():
        for i in range(df.shape[0]):
            year = int(df.loc[i, 'Year'])
            if year < 1980:
                break
            
            # batters used that year
            question = f'How many number of players are used as fielders in games in {year} in {league}?'
            correct = int(df[df['Year'] == str(year)]['#Fld'])
            options = random.sample(list(range(int(correct * 0.6), int(correct * 1.4))), k = wrong_count + 1)
            wrongs = [opt for opt in options if opt != correct][:wrong_count]
            tags = tags_ori + []
            question_list.append([question] + [correct] + wrongs + [tags])
            
            # batters used that year
            question = f'How many Stolen Bases are in {year} in {league}?'
            correct = int(df[df['Year'] == str(year)]['SB'])
            options = random.sample(list(range(int(correct * 0.6), int(correct * 1.4))), k = wrong_count + 1)
            wrongs = [opt for opt in options if opt != correct][:wrong_count]
            tags = tags_ori + []
            question_list.append([question] + [correct] + wrongs + [tags])
            
            # strikeouts used that year
            question = f'How many Defensive Chances are in {year} in {league}?'
            correct = int(df[df['Year'] == str(year)]['Ch'])
            options = random.sample(list(range(int(correct * 0.6), int(correct * 1.4))), k = wrong_count + 1)
            wrongs = [opt for opt in options if opt != correct][:wrong_count]
            tags = tags_ori + []
            question_list.append([question] + [correct] + wrongs + [tags])
            
            # stolen bases used that year
            question = f'How many Errors Committed (Fielding) are in {year} in {league}?'
            correct = int(df[df['Year'] == str(year)]['E'])
            options = random.sample(list(range(int(correct * 0.6), int(correct * 1.4))), k = wrong_count + 1)
            wrongs = [opt for opt in options if opt != correct][:wrong_count]
            tags = tags_ori + []
            question_list.append([question] + [correct] + wrongs + [tags])
            
            

National League Wins Team Wins
National League Wins Team Wins
American League Wins Team Wins
National League Batting League Year-By-Year Batting--Averages
National League Batting League Year-By-Year Batting--Totals
American League Batting League Year-By-Year Batting--Averages
American League Batting League Year-By-Year Batting--Totals
National League Pitching League Year-By-Year Pitching--Averages
National League Pitching League Year-By-Year Pitching--Totals
American League Pitching League Year-By-Year Pitching--Averages
American League Pitching League Year-By-Year Pitching--Totals
National League Fielding League Year-By-Year Fielding--Averages
National League Fielding League Year-By-Year Fielding--Totals
American League Fielding League Year-By-Year Fielding--Averages
American League Fielding League Year-By-Year Fielding--Totals
National League Misc League Year-By-Year Miscellaneous
American League Misc League Year-By-Year Miscellaneous


In [60]:
len(question_list)

2091

In [63]:
league_index_questions = pd.DataFrame(question_list, columns=columns)
with open('questions_league_index.pkl', 'wb') as f:
    pickle.dump(league_index_questions, f)
with open('questions_league_index.pkl', 'rb') as f:
    league_index_questions = pickle.load(f)

In [62]:
question_list[100]

['How many games did COL win in 2014 in National League?',
 162,
 178,
 148,
 127,
 105,
 ['Totals']]